jupyter notebook which goes through data cleaning 

In [2]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession, functions as F
import os
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/20 01:07:47 WARN Utils: Your hostname, DESKTOP-RBVA59Q resolves to a loopback address: 127.0.1.1; using 172.26.88.196 instead (on interface eth0)
24/09/20 01:07:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/20 01:07:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/20 01:07:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [128]:
df = spark.read.parquet('../data/raw/property_data.parquet')

In [5]:
df.show(5)

+--------------------+--------------------+-----------------+-----------+--------------------+--------------------+
|                name|           cost_text|            rooms|    parking|                desc|   __index_level_0__|
+--------------------+--------------------+-----------------+-----------+--------------------+--------------------+
|667 Glenhuntly Ro...|$38,000 p.a. Incl...|               []|         []|Can you hear it? ...|https://www.domai...|
|6501/35 Queensbri...|          $12,500 pw|[4 Beds, 4 Baths]|[2 Parking]|Unparalleled spac...|https://www.domai...|
|7 Jeffcott Street...|           $9,999 pw|[5 Beds, 5 Baths]|[5 Parking]|This ad is for Of...|https://www.domai...|
|5604/1 Queensbrid...|           $5,750.00|[2 Beds, 2 Baths]|[2 Parking]|Luxurious 2-bedro...|https://www.domai...|
|14A Ethel Street,...|Fully Furn - $500...|[4 Beds, 4 Baths]|[2 Parking]| class="css-dxogl...|https://www.domai...|
+--------------------+--------------------+-----------------+-----------

In [6]:
print(df.columns)

['name', 'cost_text', 'rooms', 'parking', 'desc', '__index_level_0__']


In [11]:
df.first()['cost_text']

'$38,000 p.a. Incl. Outgoings + GST'

In [13]:
small_df = df.limit(20)
small_df.show()

+--------------------+--------------------+-----------------+-----------+--------------------+--------------------+
|                name|           cost_text|            rooms|    parking|                desc|   __index_level_0__|
+--------------------+--------------------+-----------------+-----------+--------------------+--------------------+
|667 Glenhuntly Ro...|$38,000 p.a. Incl...|               []|         []|Can you hear it? ...|https://www.domai...|
|6501/35 Queensbri...|          $12,500 pw|[4 Beds, 4 Baths]|[2 Parking]|Unparalleled spac...|https://www.domai...|
|7 Jeffcott Street...|           $9,999 pw|[5 Beds, 5 Baths]|[5 Parking]|This ad is for Of...|https://www.domai...|
|5604/1 Queensbrid...|           $5,750.00|[2 Beds, 2 Baths]|[2 Parking]|Luxurious 2-bedro...|https://www.domai...|
|14A Ethel Street,...|Fully Furn - $500...|[4 Beds, 4 Baths]|[2 Parking]| class="css-dxogl...|https://www.domai...|
|28A/300 Point Coo...|                5000|[2 Beds, 2 Baths]|[6 Parking]

In [126]:
updated_small_df = small_df.withColumn(
    # see if property is furnished
    'furnished',
    F.when(F.lower(F.col('cost_text')).contains('furnished') \
           | F.lower(F.col('cost_text')).contains('furnish') \
           | F.lower(F.col('cost_text')).contains('furnishing'), True).otherwise(False)
).withColumn(
    # see if property is priced per annum
    'contains_pa',
    F.when(F.lower(F.col('cost_text')).contains('p.a') \
        | F.lower(F.col('cost_text')).contains('pa') \
        | F.lower(F.col('cost_text')).contains('per annum') \
        | F.lower(F.col('cost_text')).contains('per_annum') \
        | F.lower(F.col('cost_text')).contains('p/a'), True ).otherwise(False)

).withColumn(
    # grab price
    'cost',
    F.regexp_extract(F.col("cost_text"), r"(\$*\d+(?:,\d*)*(?:\.\d*)?)", 1)
).withColumn(
    # remove $ and ,
    'cost',
    F.regexp_replace(F.col("cost_text"), r"[$,]", "").cast("float")
).withColumn(
    # if property is priced per annum, divide by 52
    'cost',
    F.when(F.col('contains_pa') == True, F.round(F.col('cost_text') / 52, 2)).otherwise(F.col('cost_text'))
).drop('contains_pa')

In [127]:
updated_small_df.show()

24/09/20 13:40:38 ERROR Executor: Exception in task 0.0 in stage 163.0 (TID 128)
org.apache.spark.SparkFileNotFoundException: File file:/home/ulizuli/real_estate_data/real estate data/project-2-group-real-estate-industry-project-22/data/scraped/property_data.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:781)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at org.apache.spark.sql.exec

Py4JJavaError: An error occurred while calling o1428.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 163.0 failed 1 times, most recent failure: Lost task 0.0 in stage 163.0 (TID 128) (172.26.88.196 executor driver): org.apache.spark.SparkFileNotFoundException: File file:/home/ulizuli/real_estate_data/real estate data/project-2-group-real-estate-industry-project-22/data/scraped/property_data.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:781)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources

+--------------------+---------+-----------------+-----------+--------------------+--------------------+---------+
|                name|cost_text|            rooms|    parking|                desc|   __index_level_0__|furnished|
+--------------------+---------+-----------------+-----------+--------------------+--------------------+---------+
|6501/35 Queensbri...|  12500.0|[4 Beds, 4 Baths]|[2 Parking]|Unparalleled spac...|https://www.domai...|    false|
|7 Jeffcott Street...|   9999.0|[5 Beds, 5 Baths]|[5 Parking]|This ad is for Of...|https://www.domai...|    false|
|5604/1 Queensbrid...|   5750.0|[2 Beds, 2 Baths]|[2 Parking]|Luxurious 2-bedro...|https://www.domai...|    false|
|14A Ethel Street,...|   5000.0|[4 Beds, 4 Baths]|[2 Parking]| class="css-dxogl...|https://www.domai...|    false|
|28A/300 Point Coo...|   5000.0|[2 Beds, 2 Baths]|[6 Parking]|              AAAAAA|https://www.domai...|    false|
|G01/16 Kurneh Pla...|   4500.0|[3 Beds, 3 Baths]|[3 Parking]|Partially furnish.

+--------------------+---------+-----------------+-------+--------------------+--------------------+---------+----+-----+
|                name|cost_text|            rooms|parking|                desc|   __index_level_0__|furnished|beds|baths|
+--------------------+---------+-----------------+-------+--------------------+--------------------+---------+----+-----+
|6501/35 Queensbri...|  12500.0|[4 Beds, 4 Baths]|      2|Unparalleled spac...|https://www.domai...|    false|   4|    4|
|7 Jeffcott Street...|   9999.0|[5 Beds, 5 Baths]|      5|This ad is for Of...|https://www.domai...|    false|   5|    5|
|5604/1 Queensbrid...|   5750.0|[2 Beds, 2 Baths]|      2|Luxurious 2-bedro...|https://www.domai...|    false|   2|    2|
|14A Ethel Street,...|   5000.0|[4 Beds, 4 Baths]|      2| class="css-dxogl...|https://www.domai...|    false|   4|    4|
|28A/300 Point Coo...|   5000.0|[2 Beds, 2 Baths]|      6|              AAAAAA|https://www.domai...|    false|   2|    2|
|G01/16 Kurneh Pla...|  

'https://www.domain.com.au/24b-kanowna-street-williamstown-vic-3016-16572886'